# Real Estate Transactions Exploratory Data Analysis

## Environment Management

Please read the project's README for instructions on how to set up the project's environment on your computer. 

In [ ]:
# Make sure Jupiter runs at the correct environment to avoid conflicts:
import sys
sys.executable

## Import Libraries

In [ ]:
import numpy as np  # For numerical operations and arrays.	
import pandas as pd  # For data manipulation and analysis.	
import matplotlib.pyplot as plt  # For basic plotting.	
import seaborn as sns  # For enhanced plotting.	

## Data Cleaning

In [ ]:
# Import the excel file skipping the first 3 rows containing irrelevant information:
data = pd.read_excel("Original.xlsx", skiprows=3)

In [ ]:
data.head(10)

In [ ]:
# Fix the column names:
data.columns = data.iloc[0]

In [ ]:
# Investigate:
print(data.info())
data.head(10)

In [ ]:
# Drop the first row and any columns containing only NaN values.
data = data.iloc[1:].dropna(axis=1, how="all")

In [ ]:
# Investigate further:
data.info()

In [ ]:
# Drop the second 'nan' column and reset the index:
data = data.dropna(axis=1, thresh=2).reset_index(drop=True)

In [ ]:
# The final column number should be 25:
print(data.info())
data.head(10)

The data preparation is completed allowing to proceed with further analysis.

In [ ]:
print("ID Column Categories:", len(data["ID"].value_counts()), "ID Column NaN Values:", data["ID"].isna().any(), 
     "Customer ID Column Categories:", len(data["Customer ID"].value_counts()), "Customer ID Column NaN Values:", data["Customer ID"].isna().any(),
     "Property Column Categories:", len(data["Property #"].value_counts()), "Property # Column NaN Values:", data["Property #"].isna().any())

The columns "ID", "Customer ID", and "Property #" appear to have high cardinality and may not be unique identifiers based on the available data. It's difficult to completely determine their purpose without additional context. However, "ID" likely represents a sale ID, while "Customer ID" clearly indicates customer identification. The meaning of "Property #" remains unclear.

In [ ]:
# Check the duplicate IDs information:
duplicate_id = data["ID"].value_counts()[lambda x: x > 1].index
duplicate_id_indices = data[data["ID"].isin(duplicate_id)].index
data.loc[duplicate_id_indices].sort_values("ID")

A significant number of missing values make it difficult to draw conclusions from the data. There are a lot of missing values which do not allow for comparisons. The ID column probably represents a sale ID because of the information of 1009 ID, which suggests there are two different houses sold at the same date. In addition, the property number as well as the area of the houses are different which suggests the same thing.

In [ ]:
# Change dtype in order for the code below to work:
data["Property #"] = data["Property #"].astype(int)

# Check the duplicate 'Property #' column's information:
duplicate_property = data["Property #"].value_counts()[lambda x: x > 1].index
duplicate_property_indices = data[data["Property #"].isin(duplicate_property)].index
data.loc[duplicate_property_indices].sort_values("Property #").head(13)

The above cell of code doesn't help neither. The results show unambiguity. Let's proceed to another part of the data table.

In [ ]:
print("Column NaN values:", data["Building"].isna().any())
data["Building"].value_counts()

This is a categorical column that divides the buildings into distinct categories and has no missing values (NaN). The data.info() output reveals that these categories are not related to the "Type of property" column. They represent a different categorical classification and their integer type likely has no physical meaning. Since they represent categories, categorical data types are generally more memory-efficient than both object and integer types. Let's create a simple function to demonstrate this.

In [ ]:
def memory_calculation(table):
    memory = data.memory_usage().sum()
    print(f"Total Memory: {memory} bytes")
memory_calculation(table=data)

In [ ]:
# Fix the type of the column and recheck the memory:
data["Building"] = data["Building"].astype("category")
memory_calculation(table=data)

These cells were created only for testing purposes. The dtypes will be changed again later on this project.

In [ ]:
# Make some checks on some other columns:
print("'Year of sale' Non NaN Values:", data["Year of sale"].value_counts().sum(), 
      "'Month of sale' Non NaN Values:", data["Month of sale"].value_counts().sum())
print(data["Year of sale"].value_counts(), data["Month of sale"].value_counts())

In [ ]:
# Make some further checks:
print("'Type of property' Non NaN Values:", data["Type of property"].value_counts().sum())
data["Type of property"].value_counts()

In [ ]:
print("'Area (ft.)' Non NaN Values:", data["Area (ft.)"].value_counts().sum())

The units of this column are in square feets. It isn't recommended to transform them, however, we will do this for practise purposes. Let's try displaying this column in square meters meeting the SI measurement system standards.

In [ ]:
# Change the dtype of 'Area (ft.)': 
data["Area (ft.)"] = data["Area (ft.)"].astype("int")

In [ ]:
# Transform the units knowing that 1 sq feet equals 0.09290304
factor = 0.09290304

# Check the square feet mean for verification reasons:
mean_sq_feet = data["Area (ft.)"].mean().round(2)
print("Expected Square Meter Mean:", round(mean_sq_feet * factor, 2))

# Transform into square meters and check the result:
data["Area (ft.)"] = data["Area (ft.)"] * factor
mean_sq_meter = data["Area (ft.)"].mean().round(2)
print("Actual Square Meter Mean:", round(mean_sq_meter, 2))

In [ ]:
data["Area (mt.)"] = data["Area (ft.)"]
data = data.drop(columns="Area (ft.)")
data.info()

### Missing Values

In [ ]:
data.info()

19 out of 25 categories have missing values. This is the 76% of the columns. Since there isn't any more information about the data and domain expertise and since this project was created for practise reasons, we will drop some rows to see if the missing values are concentrated at the same rows. 

In [ ]:
# Drop the minimum number of rows and see what happens:
data = data.dropna(subset=["Year of sale"])

In [ ]:
data.info()

This is interesting and it means that the 72 dropped observations had missing values at 76% of their columns. Let's see what happens with the remaining missing values.

In [ ]:
# Identify rows with missing values and create a DataFrame containing only these rows: 
data_missing = data[data.isna().any(axis=1)]

# Make pandas show the initially collapsed columns:
pd.set_option("display.max_columns", 25)

data_missing

The missing values of column 'State' are missing because of the buyers foreign origin. There are no US states at foreign countries. Therefore, the missing values here means that there is no state to be provided.

In [ ]:
# Fix the 'State' column's missing values:
data["State"] = data["State"].fillna("No State")
data.info()

This is great! There are still 6 columns remaining with missing values and all these values are included at the same 17 observations. Let's check if there is a pattern between the remaining missing values.

In [ ]:
# Identify rows with missing values and create a DataFrame containing only these rows: 
data_missing = data[data.isna().any(axis=1)]
data_missing

There is a pattern in missing values related to the 'Entity' column. Both the age of the person who makes the purchase and the gender don't matter since the person represents a firm and not a human.

In [ ]:
data["Entity"].value_counts()

Therefore, all 17 observations which include missing values are firms and not individuals. However, the lack of information about the year of construction is a limitation. The final conclusion doesn't consistute a problem if we want to check some individual related patterns. Dropping these 17 observations means we will end up with 178 purchases which made by individuals which is not bad at all depending on our purposes.

In [ ]:
# Drop the firm purchases from the original data set:
data = data.dropna()
data.info()

Let's get rid of some columns. For example, the "Type of property" column includes only two offices, whereas the rest of the observations are apartment-related. Assuming we want to focus on houses, let's remove observations where the "Type of property" is "office." Subsequently, we can consider dropping low cardinality columns (with few unique values) that might not be informative for our analysis. This helps reduce dimensionality and simplify the data set.

Since this is a project for practice reasons and assuming the data table won't be needed for merging, and also assuming there is no need for customer analysis, we can further simplify the data by dropping high cardinality columns (with many unique values).

In addition, there are three columns that seem redundant and hence highly correlated: "building's age at the time of purchase," "building's year of construction," and "building's age intervals." Knowing the year the building sold, either building's age or building's year of construction is sufficient. We can drop one of these. Consequently, the "month" and "day" of construction are also be unnecessary.

In [ ]:
# Find the office observations:
office_index = data.loc[data["Type of property"] == "Office"].index

# And drop them from the data table:
data = data.drop(office_index, axis=0)

# Drop the low cardinality columns:
data = data.drop(columns=["Type of property", "Entity", "Status"])

# Drop the high cardinality columns:
data = data.drop(columns=["ID", "Property #", "Customer ID", "Name", "Surname"])

# Drop the redundant columns, both the "Interval" and the "Age at time of purchase" columns will be kept:
data = data.drop(columns=["Y", "M", "D"])

data.info()

Let's check the dtypes of the columns.

In [ ]:
data["Price"] = data["Price"].astype("float")
data[["Age at time of purchase", "Year of sale", "Month of sale"]] = data[
["Age at time of purchase", "Year of sale", "Month of sale"]
].astype("int")
data["Building"] = data["Building"].astype("str")
data.info()

In [ ]:
# Change "Age at time of purchse" column's name to make its meaning more clear:
data["Building age at time of purchase"] = data["Age at time of purchase"]
data = data.drop(columns="Age at time of purchase")
data.info()

And this is the final df.

## Exploratory Data Analysis

In [ ]:
# Manipulate the data:
bar_data = data["Interval"].value_counts()
bar_data_sorted = bar_data.index.sort_values()

# Create the bar chart:
plt.bar(bar_data_sorted, bar_data[bar_data_sorted])

# Customize bar chart:
plt.xlabel("Building's Age Intervals")
plt.ylabel("Frequency")
plt.title("Building's Age Intervals Bar Chart");

plt.savefig('barplot.png')

The distribution is spread out, with a longer tail towards the older age buildings (right skew). This suggests there's a larger variation in offers of older buildings. 

In [ ]:
data["Building age at time of purchase"].describe()

In [ ]:
# Create the histogram:
plt.hist(data["Building age at time of purchase"], edgecolor="black", bins=10)

# Create axis labels and title:
plt.xlabel("Building's Age")
plt.ylabel("Frequency")
plt.title("Distribution of the Building's Age at Time of Purchase")

# Customize the appearence of the x axis: 
custom_xticks = range(15, 80, 5)
bins = range(15, 80, 5)
plt.xticks(ticks=bins, labels=custom_xticks);

plt.savefig('histplot.png')

The histogram confirms a right-skewed distribution, with most buildings been more recent. The interquartile range (IQR) is 55 - 37 = 18 years. This means that the middle 50% age of the buildings were between 37 and 55 years old. The mean is greater than the median which is also expected since there is a positive skewness.

In [ ]:
# Create a boxplot:
plt.boxplot(data["Building age at time of purchase"], vert=False)
plt.xlabel("Building's Age")
plt.title("Distribution of the Building's Age at Time of Purchase (Boxplot)");

plt.savefig('boxplot.png')

The boxplot shows the same things have discussed already. It can be optically seen that the median is left-shifted indicating a right skew in the age distribution. The information about the IQR is not very clear. Knowing that the $IQR = 18$, this means that the outliers lie below the age of $37 - 1.5*IQR$, i.e., 10 years old and above the $Q3 + 1.5*IQR$, i.e., 82 years old. Therefore, there are no outliers inside this column. This is the reason the whiskers show the minimum and maximum osbervation age instead of the $1.5*IQR$ rule limits. The final conclusion is that the visualizations in conjunction with descriptive statistics can offer a more complete understanding.

In [ ]:
data["Price"].describe()

In [ ]:
# Scale the price:
price_scaled = data["Price"] / 1000

# Create the histogram:
plt.hist(price_scaled, edgecolor="black", bins=20)
plt.xlabel("Price in $ (X1000)")
plt.ylabel("Frequency")
plt.title("Distribution of Price")

# Fix the appearence of the x axis: 
custom_xticks = range(125, 525, 25)
bins = range(125, 525, 25)
plt.xticks(ticks=bins, labels=custom_xticks, rotation=60);

plt.savefig('histplot2.png')

The histogram reveals a positive skew. This aligns with real-world expectations, where most buyers tend to choose more affordable options. Consequently, the market offers a wider range of economical choices to meet this demand.

In [ ]:
cols_for_cor = data[["Price", "Building age at time of purchase", "Area (mt.)"]]
cols_for_cor.corr()

In [ ]:
# Create a scatterplot of Price VS Building age:
sns.scatterplot(x="Price", y="Building age at time of purchase", data=data, hue="Gender")

# Customize the scatterplot:
plt.xticks(rotation=45)
plt.xlabel("Price")
plt.ylabel("Building's Age")
plt.title("Scatter Plot: Price vs. Building's Age");

plt.savefig('scatterplot.png')

A negative correlation between the building's age and the price would expected. The scatter plot reveals what the corr() method has already revealed. There is a sligh negative correlation between the property's age and the price paid for the property.

In [ ]:
# Create a scatterplot of Price VS Area:
sns.scatterplot(x="Price", y="Area (mt.)", data=data, hue="Interval") 

# Customize the scatterplot:
plt.xticks(rotation=45)
plt.xlabel("Price")
plt.ylabel("Area in Square Meters")
plt.title("Scatter Plot: Price vs. Area in Square Meters")
plt.legend(title="Age Intervals");

plt.savefig('scatterplot2.png')

This result is also as expected.

In [ ]:
# Save the data table for future use:
data.to_csv("Real_Estate_Cleaned.csv", index=False)